In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from torch.nn.functional import relu as Relu

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

import timeit

In [34]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        ksize = 3
        netPadding = (ksize-1)//2
        self.convInput = nn.Conv2d(1, 32, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv32 = nn.Conv2d(32, 32, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv32to64 = nn.Conv2d(32, 64, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv64 = nn.Conv2d(64, 64, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv64to128 = nn.Conv2d(64, 128, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv128 = nn.Conv2d(128, 128, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv128to256 = nn.Conv2d(128, 256, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv256 = nn.Conv2d(256, 256, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv256to128 = nn.Conv2d(256, 128, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv128to64 = nn.Conv2d(128, 64, kernel_size=ksize, stride=1, padding=netPadding)
        self.conv64to1 = nn.Conv2d(64, 1, kernel_size=ksize, stride=1, padding=netPadding)
        self.pooling = nn.MaxPool2d(kernel_size=2, stride=2)
        self.upsampling = nn.Upsample(scale_factor=(2,2))

    def forward(self, x):
        # out = model(x) # Not sure what this does
        x = Relu(self.convInput(x), inplace=True)
        x = Relu(self.conv32(x), inplace=True)
        poolX = self.pooling(x)
        
        x2 = Relu(self.conv32to64(poolX), inplace=True)
        x2 = Relu(self.conv64(x2), inplace=True)
        poolX2 = self.pooling(x2)

        x3 = Relu(self.conv64(poolX2), inplace=True)
        x3 = Relu(self.conv64(x3), inplace=True)
        poolX3 = self.pooling(x3)
        
        x4 = Relu(self.conv64to128(poolX3), inplace=True)
        x4 = Relu(self.conv128(x4), inplace=True)
        poolX4 = self.pooling(x4)
        
        x5 = Relu(self.conv128to256(poolX4), inplace=True)
        
        up6 = self.upsampling(x5)
        x6 = Relu(self.conv256(up6), inplace=True)
        x6 = Relu(self.conv256(x6), inplace=True)
        poolX6 = self.pooling(x6)
        merge6 = poolX6 #torch.cat((x4,poolX6),-1)
        x6 = Relu(self.conv256(merge6), inplace=True)
        
        up7 = self.upsampling(x6)
        x7 = Relu(self.conv256(up7), inplace=True)
        x7 = Relu(self.conv256(x7), inplace=True)
        poolX7 = self.pooling(x7)
        merge7 = poolX7 #torch.cat((x3,poolX7),-1)
        x7 = Relu(self.conv256(merge7), inplace=True)
        
        up8 = self.upsampling(x7)
        x8 = Relu(self.conv256to128(up8), inplace=True)
        x8 = Relu(self.conv128(x8), inplace=True)
        poolX8 = self.pooling(x8)
        merge8 = poolX8 #torch.cat((x2,poolX8),-1)
        x8 = Relu(self.conv128(merge8), inplace=True)
        
        up9 = self.upsampling(x8)
        x9 = Relu(self.conv128to64(up9), inplace=True)
        x9 = Relu(self.conv64(x9), inplace=True)
        poolX9 = self.pooling(x9)
        merge9 = poolX9 #torch.cat((x1,poolX9),-1)
        x9 = Relu(self.conv64(merge9), inplace=True)
        
        x10 = Relu(self.conv64to1(x9), inplace=True)
        
        return x10


In [35]:
net = NeuralNet()
net.type(torch.FloatTensor)
loss_fn = nn.CrossEntropyLoss().type(torch.FloatTensor)
optimizer = optim.Adam(net.parameters(), lr=1e-2)
x = torch.randn(64, 1, 32, 32).type(torch.FloatTensor)
x_var = Variable(x.type(torch.FloatTensor))
ans = net(x_var)
ans.size()

torch.Size([64, 1, 2, 2])